In [2]:
# Import required libraries
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19, ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [3]:
image_size = (128, 128)
batch_size = 32
validation_split = 0.2
num_classes = 15

dataset_dir = r"D:\Ashraf\NCI\Semester 2\DMML2\CA2\Dataset\PlantVillage"
base_dir = r"D:\Ashraf\NCI\Semester 2\DMML2\CA2\Dataset\New"

# Function to create directories
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Split dataset into training and validation sets
def split_dataset(dataset_dir, base_dir, validation_split):
    create_dir(base_dir)
    train_dir = os.path.join(base_dir, 'train')
    validation_dir = os.path.join(base_dir, 'validation')
    create_dir(train_dir)
    create_dir(validation_dir)

    for category in os.listdir(dataset_dir):
        category_path = os.path.join(dataset_dir, category)
        if os.path.isdir(category_path):
            images = os.listdir(category_path)
            train_images, val_images = train_test_split(images, test_size=validation_split, random_state=42)

            train_category_dir = os.path.join(train_dir, category)
            val_category_dir = os.path.join(validation_dir, category)
            create_dir(train_category_dir)
            create_dir(val_category_dir)

            for image in train_images:
                shutil.copy(os.path.join(category_path, image), os.path.join(train_category_dir, image))

            for image in val_images:
                shutil.copy(os.path.join(category_path, image), os.path.join(val_category_dir, image))

split_dataset(dataset_dir, base_dir, validation_split)

In [4]:
# Data Preparation
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

validation_dataset = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

Found 16504 images belonging to 15 classes.
Found 4134 images belonging to 15 classes.


# VGG19

In [5]:
# Build VGG19 Model
vgg_base = VGG19(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
for layer in vgg_base.layers:
    layer.trainable = False
x_vgg = GlobalAveragePooling2D()(vgg_base.output)
x_vgg = Dense(1024, activation='relu')(x_vgg)
predictions_vgg = Dense(num_classes, activation='softmax')(x_vgg)
vgg_model = Model(inputs=vgg_base.input, outputs=predictions_vgg)

In [6]:
# Compile the Model
vgg_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# Train the Model
num_epochs = 5

vgg_history = vgg_model.fit(
    train_dataset,
    steps_per_epoch=train_dataset.samples // batch_size,
    epochs=num_epochs,
    validation_data=validation_dataset,
    validation_steps=validation_dataset.samples // batch_size)

Epoch 1/5


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


515/515 ━━━━━━━━━━━━━━━━━━━━ 1069s 2s/step - accuracy: 0.4796 - loss: 1.6324 - val_accuracy: 0.7226 - val_loss: 0.8462
Epoch 2/5
  1/515 ━━━━━━━━━━━━━━━━━━━━ 14:02 2s/step - accuracy: 0.6562 - loss: 0.8347

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


515/515 ━━━━━━━━━━━━━━━━━━━━ 2s 868us/step - accuracy: 0.6562 - loss: 0.8347 - val_accuracy: 0.8333 - val_loss: 0.6886
Epoch 3/5
515/515 ━━━━━━━━━━━━━━━━━━━━ 1033s 2s/step - accuracy: 0.6876 - loss: 0.9426 - val_accuracy: 0.7359 - val_loss: 0.7640
Epoch 4/5
515/515 ━━━━━━━━━━━━━━━━━━━━ 2s 606us/step - accuracy: 0.7188 - loss: 0.9039 - val_accuracy: 0.5000 - val_loss: 1.2941
Epoch 5/5
515/515 ━━━━━━━━━━━━━━━━━━━━ 864s 2s/step - accuracy: 0.7188 - loss: 0.8355 - val_accuracy: 0.7568 - val_loss: 0.7092


In [9]:
# Evaluation of VGG Model
vgg_loss, vgg_accuracy = vgg_model.evaluate(validation_dataset)

print(f"VGG Validation accuracy: {vgg_accuracy*100:.2f}%")

130/130 ━━━━━━━━━━━━━━━━━━━━ 174s 1s/step - accuracy: 0.7703 - loss: 0.6745
VGG Validation accuracy: 75.67%


In [10]:
vgg_model.save('vgg19_plant_disease_model.h5')

# Plant Care

In [17]:
import pandas as pd

care_info_df = pd.read_csv('care_info.csv')

care_info_df.head(10)

,Plant Name and Disease,Sunlight,Water,Water in liters,Cure
0,Pepper__bell___Bacterial_spot,6-8 hours of direct sunlight,Keep soil consistently moist but not waterlogged,1-1.5 liters per day,Remove infected leaves and apply copper-based ...
1,Pepper__bell___healthy,6-8 hours of direct sunlight,Water when the top 1 inch of soil is dry,1 liter per day,"No disease detected, maintain healthy growth p..."
2,Potato___Early_blight,Full sun,"Water deeply, allowing soil to dry between wat...",1-1.5 liters per day,Apply fungicides and remove affected leaves
3,Potato___Late_blight,Full sun,Keep soil evenly moist but avoid overwatering,1-1.5 liters per day,Use resistant varieties and apply appropriate ...
4,Potato___healthy,Full sun,Water when the top 2 inches of soil are dry,1-1.5 liters per day,"No disease detected, maintain healthy growth p..."
5,Tomato_Bacterial_spot,6-8 hours of direct sunlight,"Water at the base, avoiding wetting the foliage",1-1.5 liters per day,Remove infected leaves and apply copper-based ...
6,Tomato_Early_blight,6-8 hours of direct sunlight,"Water deeply, allowing soil to dry between wat...",1.5-2 liters per day,Apply fungicides and remove affected leaves
7,Tomato_Late_blight,6-8 hours of direct sunlight,Keep soil evenly moist but avoid overwatering,1.5-2 liters per day,Use resistant varieties and apply appropriate ...
8,Tomato_Leaf_Mold,6-8 hours of direct sunlight,"Water at the base, keeping foliage dry",1.5-2 liters per day,Improve air circulation and apply fungicides
9,Tomato_Septoria_leaf_spot,6-8 hours of direct sunlight,"Water at the base, avoiding wetting the foliage",1.5-2 liters per day,Remove infected leaves and apply fungicides


In [19]:
# Convert the DataFrame to a dictionary for easy lookup
care_info = care_info_df.set_index('Plant Name and Disease').T.to_dict()

# Function to get care recommendations based on predicted class
def get_care_recommendations(predicted_class):
    if predicted_class in care_info:
        return care_info[predicted_class]
    else:
        return {
            'Sunlight': 'Information not available',
            'Water': 'Information not available',
            'Water in liters': 'Information not available',
            'Cure': 'Information not available'
        }


In [20]:
# Predict and provide care recommendations for the first image in the validation set
class_labels = list(validation_dataset.class_indices.keys())
validation_dataset.reset()
vgg_predictions = vgg_model.predict(validation_dataset)
vgg_predicted_classes = np.argmax(vgg_predictions, axis=1)

# Get recommendations for the first predicted class
predicted_class = class_labels[vgg_predicted_classes[0]]
recommendations = get_care_recommendations(predicted_class)
print(f"Recommendations for {predicted_class}:")
print(f"Sunlight: {recommendations['Sunlight']}")
print(f"Water: {recommendations['Water']}")
print(f"Water in liters: {recommendations['Water in liters']}")
print(f"Cure: {recommendations['Cure']}")


130/130 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step
Recommendations for Tomato__Tomato_YellowLeaf__Curl_Virus:
Sunlight: 6-8 hours of direct sunlight
Water: Water regularly, keeping soil evenly moist
Water in liters: 1.5-2 liters per day
Cure: Remove affected plants and control whitefly population
